In [ ]:
import pandas as pd
import numpy as np
import re

tb_veic = pd.read_csv("data/dados_veiculos.csv")


Na aula de hoje vamos continuar utilizando a tabela `dados_veículos.csv`. Além de carregá-la vamos realizar uma limpeza dos nomes das colunas:

In [ ]:
pattern = r"[^a-zA-Z0-9]"
tb_veic.columns = [re.sub(pattern, "_", column.lower()) for column in tb_veic.columns]


Vamos utilizar o método `.info()` para ver o resultado:

In [ ]:
tb_veic.info()


# Data Manipulation - Group By

- `.groupby()` é uma forma de **agregar** todos os resultados para cada chave única
- Sempre que você faz uma **agregação**, o resultado final terá 1 linha para cada valor pelo qual você agregou, portanto, é obrigatório que se aplique uma função agregadora para que todos os valores sejam sumarizados em um único valor associado àquela chave.

Por exemplo, se tivermos:

Fruta | Quantidade
-----|----:
Laranja | 10
Maçã | 10
Abacate | 2
Abacate | 1
Melancia  | 1
Laranja  | 4

O resultado de um `.groupby` por 'Fruta' resultaria em 4 linhas

Fruta | Função Agregadora
----|-----
Laranja | ?
Maçã | ?
Abacate | ?
Melancia | ?

O `?` representa o valor agregado. Temos, obrigatoriamente, que sumarizar os dados relacionados àquele registro em um único dado. Para isso, podemos fazer a média, soma, contagem, ou qualquer outra função agregadora.

Fruta | soma(Quantidade)
----|----:
Laranja | 14
Maçã | 10
Abacate | 3
Melancia | 1

**Funções agregadora/Métodos**

- `.mean()`
- `.median()`
- `.max()`
- `.min()`
- `.sum()`
- `.count()`
- `.describe()`
- `.agg()`
- Uma função sua!

## Chave Única e Função Única

Vamos começar com o tipo mais simples de agregação: utilizando apenas uma coluna como **chave** e uma função como **agregação**. Vamos começar analisando a média de nossas variáveis numéricas, utilizando como chave a coluna `drivetrain` (*transmissão*).

Para isso utilizaremos o método `.groupby()` - um método dos DataFrames. Para utilizarmos este método, vamos analisar sua sintaxe:

* `tb_veic` - tabela que vamos agregar
* `.groupby(by = 'drivetrain')` - método para criar os grupos de **chaves**, especificada no argumento `by`
* `.mean()` - método para agregar as variáveis restantes (que não compõe a chave), neste caso através da média.

Juntando tudo temos:

In [ ]:
tb_veic.groupby(by="drivetrain").mean()


Vamos testar a mesma **chave** mas utilizando o método de **agregação** `.count()`, que calcula o # de linhas com valores não-nulos.

In [ ]:
tb_veic.groupby(by="drivetrain").count()


Podemos guardar o resultado de uma agregação em uma variável: o resultado (após aplicação da função de agregação) é um `DataFrame` onde os índices (*nome das linhas*) são os valores da chave e cada coluna é o resultado da aplicação da função de agregação.

In [ ]:
tb_agg_drivetrain = tb_veic.groupby(by="drivetrain").count()
type(tb_agg_drivetrain)


In [ ]:
tb_agg_drivetrain.head()


In [ ]:
tb_agg_drivetrain.loc["4-Wheel or All-Wheel Drive", :]


### Agregando apenas uma variável

No exemplo acima, ao utilizarmos os métodos `.mean()` e `.median()` recebemos de volta uma agregação de todas as variáveis numéricas (no caso, todas as variáveis para as quais estes métodos funcionam).

No entanto, muitas vezes queremos agregar apenas uma (ou algumas) variáveis. Vamos ver como fazer isso de uma maneira simples, calculando o número de ciclindros médio por tipo de tração:

In [ ]:
tb_veic.groupby(by="drivetrain")["cylinders"].mean()


Um método muito utilizado após a agregação é o `.reset_index()` que transforma o índice da chave de volta em coluna:

In [ ]:
tb_veic.groupby(by="drivetrain")["cylinders"].mean().reset_index()


Podemos utilizar um iterável no lugar do string `"cylinders"` para calcular a média de múltiplas variáveis:

In [ ]:
lista_vars = ["cylinders", "city_mpg"]
tb_veic.groupby(by="drivetrain")[lista_vars].mean().reset_index()


## Chave Múltipla e Função Única

As chaves especificadas no *argumento* `by = ` do método `.groupby()` não precisa ser um `string`! Podemos utilizar um iterável para criar uma chave combinada. Por exemplo, queremos analisar a média das variáveis `city_mpg` (eficiência urbana do automóvel) e `highway_mpg` (eficiência na estrada) para as diferentes combinações de tração, `drivetrain`, e # de cilíndros, `cylinders`:

In [ ]:
lista_vars = ["city_mpg", "highway_mpg"]
tb_veic.groupby(by=["drivetrain", "cylinders"])[lista_vars].mean()


O resultado é um DataFrame com um `MultIndex`: um índice com mais de um *nível*. Para cada tipo de transmissão temos todos os # de ciclindros daquela transmissão (`4`, `6` e `8` para `2-Wheel Drive` por exemplo). Para acessar diferentes linhas através de nossa chave, utilizaremos um índice em tupla:

In [ ]:
tb_agg_cyldt = tb_veic.groupby(by=["drivetrain", "cylinders"])[lista_vars].mean()
tb_agg_cyldt.loc[("2-Wheel Drive", 4), :]


Podemos utilizar o método `.reset_index()` para construir uma tabela *normal* a partir disto:

In [ ]:
tb_agg_cyldt = (
    tb_veic.groupby(by=["drivetrain", "cylinders"])[lista_vars].median().reset_index()
)
tb_agg_cyldt.head(10)


## Chave Múltipla e Múltiplas Funções

Além de incluirmos mais que uma variável como chave de nosso agrupamento, podemos utilizar mais que uma função de agregação ao mesmo tempo. Por exemplo, vamos calcular a média e a mediana para a variável `city_mpg`. Para isso utilizaremos o método `.agg()` no lugar da nossa função de agregação. Vamos passar um iterável com o **nome** de cada função de agregação que utilizaremos:

In [ ]:
tb_agg_musig_cyldt = tb_veic.groupby(by=["drivetrain", "cylinders"])["city_mpg"].agg(
    ["mean", "median"]
)
tb_agg_musig_cyldt

Vamos utilizar o método `.reset_index()` para colapsar os `MultIndex` de nossa tabela:

In [ ]:
tb_agg_cyldt = tb_veic.groupby(by=["drivetrain", "cylinders"])["city_mpg"].agg(
    ["mean", "median"]
).reset_index()
tb_agg_cyldt.head(10)

Podemos calcular múltiplas agregações de múltiplas chaves mas devemos tomar cuidado com o **nome das colunas**: ele será um `MultIndex`!

In [ ]:
tb_agg_cyldt = tb_veic.groupby(by=["drivetrain", "cylinders"])[lista_vars].agg(
    ["mean", "median"]
)
tb_agg_cyldt.head(10)

In [ ]:
tb_agg_cyldt.columns

Se utilizarmos o método `.reset_index()` teremos nossas chaves como colunas, mas o nome das colunas agregadas continuará sendo um `MultIndex`:

In [ ]:
tb_agg_cyldt = tb_veic.groupby(by=["drivetrain", "cylinders"])[lista_vars].agg(
    ["mean", "median"]
).reset_index()
tb_agg_cyldt.head(10)

In [ ]:
tb_agg_cyldt.columns

### Agregação Nomeada

A situação apresentada acima não é ideal por dois motivos:

1. Muitas vezes queremos especificar colunas específicas para cada agregação (por exemplo, a **média** de `city_mpg` e o máximo de `year`);
1. Além disso seria bom conseguir nomear explicitamente as variáveis criadas (por exemplo, `mean_city_mpg` e `max_year`).

Para resolver esses dois problemas podemos utilizar uma outra maneira de agregação: a agregação nomeada. Ao invés de passar um iterável com o nome das funções que queremos aplicar, vamos utilizar a função `pd.NamedAgg()` para construir cada coluna:

In [ ]:
tb_veic.groupby(by=["drivetrain", "cylinders"]).agg(
    mean_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc="mean"),
    max_year=pd.NamedAgg(column="year", aggfunc="max")
).reset_index()


Agora vamos utilizar os conceitos acima para tornar nossa análise da eficiência de automóveis mais robusta: vamos incluir, além da média de `city_mpg` e `highway_mpg`, o # de observações em cada grupo (utilizando a `aggfunc = "count"`) e o desvio padrão das duas variáveis de eficiência (utilizando `aggfunc = "std"`):

In [ ]:
tb_agg_veic = (
    tb_veic.groupby(by=["drivetrain", "cylinders"])
    .agg(
        contagem=pd.NamedAgg(column="year", aggfunc="count"),
        avg_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc="mean"),
        std_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc="std"),
        avg_highway_mpg=pd.NamedAgg(column="highway_mpg", aggfunc="mean"),
        std_highway_mpg=pd.NamedAgg(column="highway_mpg", aggfunc="std"),
    )
    .reset_index()
)

In [ ]:
tb_agg_veic.describe()

In [ ]:
tb_agg_veic_robusto = tb_agg_veic[tb_agg_veic['contagem'] > 100]

In [ ]:
tb_agg_veic_robusto

# Data Manipulation - Joins

* Como unir DataFrames através de colunas
* [Fruits Example](https://docs.google.com/spreadsheets/d/1foV1THfhVc5WduBTMjpxmthAD1BDHS9FUpH-WgGDH5E/edit?usp=sharing)

Um join pode ser classificado ao longo de dois eixos:

1. **Direção**: Esquerda, direita ou interno (left, right e inner);
2. **Cardinalidade**: Um para um, um para muitos, muitos para muitos (1:1, 1:n, n:n).

https://towardsdatascience.com/can-we-stop-with-the-sql-joins-venn-diagrams-insanity-16791d9250c3

## Tipos de Join

Vamos analisar cada tipo de join através de sua cardinalidade

### Um para Um (1:1)

Um join um pra um ocorre quando a **relação entre as chaves** é 1:1, ou seja, para cada chave particular na tabela A teremos apenas uma chave equivalente na tabela B.

Esse é o tipo *desejado* **mais comum de join**, e serve para cruzarmos tabelas onde cada linha representa o **o mesmo tipo de observação** (em termos técnicos, tabelas com a mesma chave única). Por exemplo, se temos uma tabela de informações geográficas por município e outra de informações demográficas podemos utilizar um join 1:1 (usando o município) para termos uma tabela de informações demográficas e geográficas.

In [ ]:
nome_frutas = ["Uva", "Abacate", "Melancia", "Atemoia", "Laranja"]
producao_frutas = [10, 1, 1, 3, 4]
tb_prod_frutas = pd.DataFrame(
    {"nome_fruta": nome_frutas, "producao_kg": producao_frutas}
)
tb_prod_frutas


In [ ]:
fruta = ["Uva", "Abacate", "Melancia", "Carambola"]
cidade = ["Campinas", "Campinas", "Atibaia", "Rio de Janeiro"]
tb_pomar_cidade = pd.DataFrame({"nome_fruta": fruta, "cidade": cidade})
tb_pomar_cidade


#### Inner Join (o que é comum às duas tabelas)

In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, on="nome_fruta", how="inner")


#### Left Join (tudo o que está na tabela a esquerda)

In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, on="nome_fruta", how="left")


#### Right Join (tudo o que está na tabela a direita)

In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, on="nome_fruta", how="right")


#### Chaves com nomes distintos

Embora seja considerado boa prática que todas as colunas de chave tenham nomes uniformes, muitas vezes vamos nos deparar com situações onde queremos unir duas tabelas através de um chave que está em colunas com nomes diferentes. 

Para tratar isso utilizaremos os argumentos `left_on =` e `right_on =`:

In [ ]:
nome_frutas = ["Uva", "Abacate", "Melancia", "Atemoia", "Laranja"]
producao_frutas = [10, 1, 1, 3, 4]
tb_prod_frutas = pd.DataFrame(
    {"nome_fruta": nome_frutas, "producao_kg": producao_frutas}
)
tb_prod_frutas


In [ ]:
fruta = ["Uva", "Abacate", "Melancia", "Carambola"]
cidade = ["Campinas", "Campinas", "Atibaia", "Rio de Janeiro"]
tb_pomar_cidade = pd.DataFrame({"nome_pomar_fruta": fruta, "cidade": cidade})
tb_pomar_cidade


In [ ]:
pd.merge(
    tb_prod_frutas, tb_pomar_cidade, left_on="nome_fruta", right_on="nome_pomar_fruta"
)


### Um para N (1:N)

A relação 1:N é *quase* tão comum quanto a relação 1:1 e, em geral, surge quando queremos unir uma tabela com uma chave que é **parte** da chave da segunda tabela. Por exemplo, se nossa tabela de informações demográficas do exemplo anterior utilize a chave Município-Mês, o join com a tabela geográfica (chave Município) será 1:N.

Uma situação prática é quando temos informações de clientes que queremos cruzar com informações de pedidos: todo pedido tem apenas um cliente, cada cliente tem múltiplos pedidos.

In [ ]:
nome_frutas = ["Uva", "Abacate", "Melancia", "Atemoia", "Laranja"]
preco_frutas = [10, 5, 3, 30, 2]
tb_prod_frutas = pd.DataFrame(
    {"nome_fruta": nome_frutas, "preco_kg": preco_frutas}
)
tb_prod_frutas

In [ ]:
fruta = ["Uva", "Abacate", "Melancia", "Uva", "Carambola"]
cidade = ["Campinas", "Campinas", "Atibaia", "Atibaia", "Rio de Janeiro"]
qtd_prod = [1000, 5000, 2500, 300, 150]
tb_pomar_cidade = pd.DataFrame({"nome_fruta": fruta, "cidade": cidade, "qtd_produzida" : qtd_prod})
tb_pomar_cidade


In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, on="nome_fruta")


### M para N (M:N)

Os joins N para N são mais raros que os outros dois tipos (exceto quando erramos e nosso join 1:1 vira um N:N...). Ele ocorre quando queremos unir duas tabelas por uma chave que **não é única** nas duas tabelas.

Um exemplo comum é quando queremos cruzar informações de produtos e clientes: como cada cliente pode comprar múltiplos produtos diferentes e cada produto pode ser comprado por múltiplos clientes a relação é uma relação M:N.

Tanto no Pandas quanto no SQL raramente criaremos relações M:N - elas são mediadas por uma terceira chave (o código do pedido no exemplo acima). No entanto, **esse tipo de join ocorre MUITO quando fazemos algo errado**, não conhecer as chaves de nossa tabela por exemplo!

In [ ]:
nome_frutas = ["Uva", "Uva", "Abacate", "Melancia", "Atemoia", "Laranja"]
producao_frutas = [10, 5, 1, 1, 3, 4]
tb_prod_frutas = pd.DataFrame(
    {"nome_fruta": nome_frutas, "producao_kg": producao_frutas}
)
tb_prod_frutas


In [ ]:
fruta = ["Uva", "Abacate", "Melancia", "Uva", "Carambola"]
cidade = ["Campinas", "Campinas", "Atibaia", "Atibaia", "Rio de Janeiro"]
tb_pomar_cidade = pd.DataFrame({"nome_fruta": fruta, "cidade": cidade})
tb_pomar_cidade


In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, on="nome_fruta")


## Chaves Compostas

As chaves de nossos joins não precisam estar localizadas apenas em uma coluna: podemos ter partes diferentes de uma chave em colunas diferentes (por exemplo, em uma tabela cuja chave seja Município-Ano podemos ter uma coluna Município e outra Ano).

Para realizar joins com chaves compostas precisaremos alterar um pouco como especificamos o argumento `on`.

In [ ]:
nome_frutas = ["Uva", "Abacate", "Melancia", "Atemoia", "Laranja"]
cidade = ["Campinas", "Atibaia", "Atibaia", "Rio de Janeiro", "Rio de Janeiro"]
preco_frutas = [10, 5, 3, 30, 2]
tb_prod_frutas = pd.DataFrame(
    {"nome_fruta": nome_frutas, "cidade" : cidade, "preco_kg": preco_frutas}
)
tb_prod_frutas

In [ ]:
fruta = ["Uva", "Abacate", "Melancia", "Uva", "Carambola"]
cidade = ["Campinas", "Campinas", "Atibaia", "Atibaia", "Rio de Janeiro"]
qtd_prod = [1000, 5000, 2500, 300, 150]
tb_pomar_cidade = pd.DataFrame({"nome_fruta": fruta, "cidade": cidade, "qtd_produzida" : qtd_prod})
tb_pomar_cidade


In [ ]:
tb_prod_frutas.merge(tb_pomar_cidade, on = ['nome_fruta', 'cidade'])

# Bonus - Análise de Eficiência de Automóveis

Vamos utilizar os conceitos vistos hoje para responder a pergunta *Será que a eficiência dos carros aumentou ao longo dos anos?*.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
tb_agg_year = tb_veic.groupby(by="year").agg(
    mean_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc="mean"),
    mean_highway_mpg=pd.NamedAgg(column="highway_mpg", aggfunc="mean"),
)
plt.plot(tb_agg_year["mean_city_mpg"])


**Quais conclusões podemos tirar do gráfico acima? Como podemos melhorar nossa análise?**

In [ ]:
tb_agg_cyl = (
    tb_veic.groupby(by="cylinders")
    .agg(
        mean_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc="mean"),
        mean_highway_mpg=pd.NamedAgg(column="highway_mpg", aggfunc="mean"),
    )
    .reset_index()
)
tb_agg_cyl


In [ ]:
tb_veic = pd.merge(tb_veic, tb_agg_cyl, on="cylinders")
tb_veic


In [ ]:
tb_veic["norm_city_mpg"] = tb_veic["city_mpg"] / tb_veic["mean_city_mpg"]
tb_veic["norm_highway_mpg"] = tb_veic["highway_mpg"] / tb_veic["mean_highway_mpg"]
tb_veic


In [ ]:
tb_agg_year = tb_veic.groupby(by="year").agg(
    mean_city_mpg=pd.NamedAgg(column="norm_city_mpg", aggfunc="mean"),
    mean_highway_mpg=pd.NamedAgg(column="norm_highway_mpg", aggfunc="mean"),
    contagem=pd.NamedAgg(column="norm_highway_mpg", aggfunc="count"),
)
plt.plot(tb_agg_year["mean_city_mpg"])
# plt.plot(tb_agg_year['contagem'])


In [ ]:
tb_agg_yearcyl = (
    tb_veic.groupby(["cylinders", "year"])
    .agg(
        mean_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc="mean"),
        n_linhas=pd.NamedAgg(column="city_mpg", aggfunc="count"),
    )
    .reset_index()
)
tb_agg_yearcyl


In [ ]:
tb_agg_yearcyl["last_year"] = tb_agg_yearcyl["year"] - 1
tb_agg_yearcyl


In [ ]:
tb_agg_yearcyla = pd.merge(
    tb_agg_yearcyl,
    tb_agg_yearcyl,
    left_on=["cylinders", "last_year"],
    right_on=["cylinders", "year"],
    suffixes=("", "_ly"),
)
tb_agg_yearcyla[tb_agg_yearcyla["cylinders"] == 6]


In [ ]:
tb_agg_yearcyla["var_anual"] = (
    tb_agg_yearcyla["mean_city_mpg"] - tb_agg_yearcyla["mean_city_mpg_ly"]
) / tb_agg_yearcyla["mean_city_mpg_ly"]


In [ ]:
aa = tb_agg_yearcyla.groupby(["cylinders"]).agg(
    var_anual_media=pd.NamedAgg("var_anual", "mean"),
    min_ano=pd.NamedAgg("year", "min"),
    max_ano=pd.NamedAgg("year", "max"),
    n_linhas=pd.NamedAgg("n_linhas", "sum"),
)
aa[aa["n_linhas"] > 149]


# Bonus - Bins Concept

![Imgs](https://i.stack.imgur.com/pObHa.png)


### two methods: cut vs qcut

The major distinction is that qcut will calculate the size of each bin in order to make sure the distribution of data in the bins is equal. In other words, all bins will have (roughly) the same number of observations but the bin range will vary.

On the other hand, cut is used to specifically define the bin edges. There is no guarantee about the distribution of items in each bin. In fact, you can define bins in such a way that no items are included in a bin or nearly all items are in a single bin.

In [ ]:
pd.cut(tb_veic["city_mpg"], bins=10)


In [ ]:
tb_veic["classifA_city_mpg"] = pd.cut(tb_veic["city_mpg"], bins=10)
tb_veic["classifA_city_mpg"].value_counts()


In [ ]:
tb_veic["classifB_city_mpg"] = pd.qcut(tb_veic["city_mpg"], q=10)
tb_veic["classifB_city_mpg"].value_counts()


# Bonus - Categorical Data Into Columns

>    - dummies
>    - One hot encoding

In [ ]:
pd.get_dummies(tb_veic["classifB_city_mpg"])


In [ ]:
dummies = pd.get_dummies(tb_veic["classifB_city_mpg"])
pd.concat([tb_veic, dummies], axis=1)


# Bonus - Funções Customizadas de Agregação

In [ ]:
def dist_iq(panda_series):
    if len(panda_series) > 5:
        return (
            panda_series.quantile(0.75) - panda_series.quantile(0.25)
        ) / panda_series.quantile(0.5)
    else:
        return np.nan

In [ ]:
tb_veic.groupby(by=["drivetrain", "cylinders"]).agg(
    avg_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc="mean"),
    std_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc="std"),
    iqn_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc=dist_iq),
)
